# 02: Multimodal Requests & Streaming

In [ ]:
from anthropic import Anthropic

In [9]:
import os
if not os.environ["ANTHROPIC_API_KEY"]:
    print("environment variable ANTHROPIC_API_KEY not found, checking in api.txt")
    with open("api.txt") as i:
        api = i.read().strip()
        if not api:
            print("nothing found in api.txt, create a key in console.anthropic.com and paste it there")
        else:
            os.environ["ANTHROPIC_API_KEY"] = api
            print("key found in api.txt")  
else:
    print("environment variable ANTHROPIC_API_KEY found")    

environment variable ANTHROPIC_API_KEY not found, checking in api.txt
key found in api.txt


In [ ]:
client = Anthropic()
MODEL_NAME="claude-3-5-sonnet-20241022"

## Content Blocks

In [ ]:
messages = [
    {
        "role": "user",
        "content": "tell me a joke"
    }
]

response = client.messages.create(
    messages=messages,
    model=MODEL_NAME,
    max_tokens=200
)
print(response.content[0].text)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "tell me a joke"},
        ]
    }
]

response = client.messages.create(
    messages=messages,
    model=MODEL_NAME,
    max_tokens=200
)
print(response.content[0].text)

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "who"},
            {"type": "text", "text": "made"},
            {"type": "text", "text": "you?"},
        ]
    }
]
response = client.messages.create(
    messages=messages,
    model=MODEL_NAME,
    max_tokens=200
)
print(response.content[0].text)

## Image Prompts

In [ ]:
from IPython.display import Image
Image(filename='images/food.png') 

## Image Messages

In [ ]:
import base64
# opens the image file in "read binary" mode
with open("images/food.png", "rb") as image_file:
    #reads the contents of the image as a bytes object
    binary_data = image_file.read() 
    #encodes the binary data using Base64 encoding
    base_64_encoded_data = base64.b64encode(binary_data) 
    #decodes base_64_encoded_data from bytes to a string
    base64_string = base_64_encoded_data.decode('utf-8')

In [ ]:
base64_string[:100]

In [ ]:
messages = [
    {
        "role": "user",
        "content": [{
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/png",
                "data": base64_string
            },
        },
        {
            "type": "text",
            "text": """How many to-go containers of each type 
            are in this image?"""
        }]
    }
]

In [ ]:
response = client.messages.create(
    messages=messages,
    model=MODEL_NAME,
    max_tokens=200
)
print(response.content[0].text)

## Image Block Helper Function

In [ ]:
import base64
import mimetypes

def create_image_message(image_path):
    # Open the image file in "read binary" mode
    with open(image_path, "rb") as image_file:
        # Read the contents of the image as a bytes object
        binary_data = image_file.read()
    # Encode the binary data using Base64 encoding
    base64_encoded_data = base64.b64encode(binary_data)
    # Decode base64_encoded_data from bytes to a string
    base64_string = base64_encoded_data.decode('utf-8')
    # Get the MIME type of the image based on its file extension
    mime_type, _ = mimetypes.guess_type(image_path)
    # Create the image block
    image_block = {
        "type": "image",
        "source": {
            "type": "base64",
            "media_type": mime_type,
            "data": base64_string
        }
    }
    
    
    return image_block

In [ ]:
Image("images/plant.png")

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            create_image_message("./images/plant.png"),
            {"type": "text", "text": "What species is this?"}
        ]
    }
]

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

## A Real World Use Case

In [ ]:
Image(filename='images/invoice.png') 

In [ ]:
messages = [
    {
        "role": "user",
        "content": [
            create_image_message("./images/invoice.png"),
            {"type": "text", "text": """
                Generate a JSON object representing the contents
                of this invoice.  It should include all dates,
                dollar amounts, and addresses. 
                Only respond with the JSON itself.
            """
            }
        ]
    }
]

response = client.messages.create(
    model=MODEL_NAME,
    max_tokens=2048,
    messages=messages
)
print(response.content[0].text)

## Streaming

In [ ]:
response = client.messages.create(
    max_tokens=1024,
    messages=[{"role": "user", "content": "write a poem"}],
    model=MODEL_NAME,
)
print(response.content[0].text)

In [ ]:
with client.messages.stream(
    max_tokens=1024,
    messages=[{"role": "user", "content": "write a poem"}],
    model=MODEL_NAME,
) as stream:
  for text in stream.text_stream:
      print(text, end="", flush=True)